In [1]:
import pandas as pd
from correction.compute import *
from correction.helpers import *

# Prepare your dataset
The core code use different functions for correct the light curve, in the following blocks we'll use theses functions for correct a data examples.

In [2]:
detections = pd.read_parquet("../data_examples/parquets/10_objects_detections.parquet")
display(detections.head())

non_detections = pd.read_parquet("../data_examples/parquets/10_objects_non_detections.parquet", 
                                 columns=["objectId", "fid", "jd", "diffmaglim"])
display(non_detections.head())

,objectId,jd,fid,pid,diffmaglim,pdiffimfilename,programpi,programid,candid,isdiffpos,...,simag3,szmag3,sgscore3,distpsnr3,nmtchps,rfid,jdstartref,jdendref,nframesref,parent_candid
0,ZTF18aaioozo,2.458273e+06,1,518415875315,20.182840,ztf_20180603415822_000683_zg_c14_o_q2_scimrefd...,Kulkarni,1,518415875315010002,f,...,15.847300,15.798900,0.998750,8.699934,15,683120153,2.458205e+06,2.458231e+06,15,0
1,ZTF18aaurrwb,2.458279e+06,1,524348253515,20.939032,ztf_20180609348229_000723_zg_c09_o_q4_scimrefd...,Kulkarni,1,524348253515015051,t,...,21.393101,21.497999,0.751875,19.763674,9,723120135,2.458204e+06,2.458207e+06,16,0
2,ZTF18abadmxd,2.458282e+06,1,527431606115,20.560465,ztf_20180612431609_000804_zg_c16_o_q2_scimrefd...,Kulkarni,1,527431606115010124,f,...,16.569500,16.329300,0.974434,8.267725,37,804120161,2.458229e+06,2.458277e+06,15,0
3,ZTF18abadvod,2.458282e+06,2,527448113315,20.925859,ztf_20180612448102_000687_zr_c09_o_q2_scimrefd...,Kulkarni,1,527448113315010023,f,...,21.053900,20.278700,0.088690,4.907485,53,687120233,2.458199e+06,2.458257e+06,15,0
4,ZTF18abadvtf,2.458282e+06,2,527450573615,20.881939,ztf_20180612450556_000730_zr_c10_o_q1_scimrefd...,Kulkarni,1,527450573615015012,t,...,20.822399,20.271200,0.625548,7.855210,34,730120236,2.458223e+06,2.458280e+06,15,0


,objectId,fid,jd,diffmaglim
369,ZTF18aazeood,2,2.458384e+06,20.273001
137,ZTF18aaioozo,1,2.458661e+06,20.883699
449,ZTF18abadmxd,2,2.458313e+06,20.606800
394,ZTF18abadmxd,2,2.458321e+06,20.769300
347,ZTF18aazeood,1,2.458434e+06,19.930201


First, we need a modified julian dates.

In [3]:
detections["mjd"] = detections.jd - 2400000.5
del detections["jd"]

non_detections["mjd"] = non_detections.jd - 2400000.5
del non_detections["jd"]

2020-06-23 17:58:37 INFO numexpr.utils._init_num_threads: NumExpr defaulting to 4 threads.


# Getting a LC corrected
Apply the `apply_correction_df` functon for all unique [objectId, fid] pairs.

In [4]:
corrected = detections.groupby(["objectId", "fid"]).apply(apply_correction_df)
display(corrected.head())

pid  diffmaglim  \
objectId     fid candid                                           
ZTF17aaaecgt 1   550436093915015009    550436093915   20.544632   
                 1209408473915010011  1209408473915   20.269983   
                 1004203723915015017  1004203723915   20.481764   
                 978289493915010013    978289493915   20.881487   
                 1009243913915015013  1009243913915   20.051744   

                                                                        pdiffimfilename  \
objectId     fid candid                                                                   
ZTF17aaaecgt 1   550436093915015009   ztf_20180705436100_000801_zg_c10_o_q4_scimrefd...   
                 1209408473915010011  ztf_20200424408472_000801_zg_c10_o_q4_scimrefd...   
                 1004203723915015017  ztf_20191002203727_000801_zg_c10_o_q4_scimrefd...   
                 978289493915010013   ztf_20190906289468_000801_zg_c10_o_q4_scimrefd...   
                 1009243913915015013  ztf_20191007243912_000801_zg_c10_o_q4_scimrefd...   

                                        programpi  programid  isdiffpos  \
objectId     fid candid                                                   
ZTF17aaaecgt 1   550436093915015009      Kulkarni          1        1.0   
                 1209408473915010011     Kulkarni          1       -1.0   
                 1004203723915015017         TESS          1        1.0   
                 978289493915010013          TESS          1       -1.0   
                 1009243913915015013  TESS_Prince          3        1.0   

                                      tblid   nid  rcid  field  ...  \
objectId     fid candid                                         ...   
ZTF17aaaecgt 1   550436093915015009       9   550    39    801  ...   
                 1209408473915010011     11  1209    39    801  ...   
                 1004203723915015017     17  1004    39    801  ...   
                 978289493915010013      13   978    39    801  ...   
                 1009243913915015013     13  1009    39    801  ...   

                                        jdstartref      jdendref  nframesref  \
objectId     fid candid                                                        
ZTF17aaaecgt 1   550436093915015009   2.458232e+06  2.458289e+06          15   
                 1209408473915010011  2.458232e+06  2.458289e+06          15   
                 1004203723915015017  2.458232e+06  2.458289e+06          15   
                 978289493915010013   2.458232e+06  2.458289e+06          15   
                 1009243913915015013  2.458232e+06  2.458289e+06          15   

                                      parent_candid           mjd  corrected  \
objectId     fid candid                                                        
ZTF17aaaecgt 1   550436093915015009               0  58304.436099       True   
                 1209408473915010011              0  58963.408472       True   
                 1004203723915015017              0  58758.203727       True   
                 978289493915010013               0  58732.289491       True   
                 1009243913915015013              0  58763.243912       True   

                                      magpsf_corr  sigmapsf_corr  \
objectId     fid candid                                            
ZTF17aaaecgt 1   550436093915015009     16.510900       0.014875   
                 1209408473915010011    17.879532       0.055861   
                 1004203723915015017    16.742756       0.009440   
                 978289493915010013     17.739686       0.052527   
                 1009243913915015013    16.697498       0.016639   

                                      sigmapsf_corr_ext  dubious  
objectId     fid candid                                           
ZTF17aaaecgt 1   550436093915015009            0.018298    False  
                 1209408473915010011           0.067333    False  
                 1004203723915015017           0.

# Getting magnitude statistics for LC corrected

In [5]:
magstats = corrected.groupby(["objectId", "fid"]).apply(apply_mag_stats)
display(magstats)

corrected  nearZTF  nearPS1  stellarZTF  stellarPS1  \
objectId     fid                                                        
ZTF17aaaecgt 1         True     True     True        True        True   
             2         True     True     True        True        True   
ZTF18aaioozo 1         True     True     True        True        True   
             2         True     True     True        True        True   
ZTF18aaurrwb 1        False    False    False        True       False   
ZTF18aazeood 1         True     True     True        True        True   
             2         True     True     True        True        True   
ZTF18abadmxd 1         True     True     True        True        True   
ZTF18abadvod 1         True     True     True        True        True   
             2         True     True     True        True        True   
ZTF18abadvtf 2        False    False    False        True       False   
ZTF18abbklxk 2         True     True     True        True        True   
ZTF18abdkytr 1         True     True     True        True       False   
             2        False    False     True        True       False   
ZTF18abgptvl 1         True     True     True        True        True   
             2         True     True     True        True        True   

                  stellar  ndet  ndubious  nrfid  magpsf_mean  ...  \
objectId     fid                                               ...   
ZTF17aaaecgt 1       True   231         0      2    18.249178  ...   
             2       True   208         3      2    16.122793  ...   
ZTF18aaioozo 1       True    98         0      2    17.819777  ...   
             2       True    60         0      2    17.756157  ...   
ZTF18aaurrwb 1      False     3         0      2    20.468546  ...   
ZTF18aazeood 1       True   367         0      4    18.172682  ...   
             2       True   396         0      4    17.584890  ...   
ZTF18abadmxd 1       True     3         0      2    18.450991  ...   
ZTF18abadvod 1       True     1         0      1    20.467178  ...   
             2       True     7         0      1    19.889450  ...   
ZTF18abadvtf 2      False     1         0      1    18.618200  ...   
ZTF18abbklxk 2       True     1         0      1    19.710394  ...   
ZTF18abdkytr 1       True     2         0      1    18.727316  ...   
             2      False     1         1      1    18.162201  ...   
ZTF18abgptvl 1       True     1         0      1    19.674820  ...   
             2       True     1         0      1    18.942438  ...   

                  magpsf_corr_first  magpsf_corr_last  magap_mean  \
objectId     fid                                                    
ZTF17aaaecgt 1            17.200124         16.553944   18.321594   
             2            14.469480         14.461061   16.190769   
ZTF18aaioozo 1            15.977781         16.000094   17.895746   
             2            15.921919         15.945003   17.805830   
ZTF18aaurrwb 1                  NaN               NaN   20.358467   
ZTF18aazeood 1            16.589251         16.185487   18.260881   
             2            15.937946         15.797239   17.641266   
ZTF18abadmxd 1            14.717818         14.633466   18.612566   
ZTF18abadvod 1            18.088278         18.088278   20.612000   
             2            17.372926         17.441555   19.964672   
ZTF18abadvtf 2                  NaN               NaN   18.618200   
ZTF18abbklxk 2            16.530550         16.530550   19.707399   
ZTF18abdkytr 1            15.715557         15.709698   18.853748   
             2                  NaN               NaN   18.302999   
ZTF18abgptvl 1            16.117216         16.117216   20.391100   
             2            15.426545         15.426545   19.070299   

                  magap_median  magap_max  magap_min  magap_first  magap_last  \
objectId     fid                                                                
ZTF17aaaecgt 1       18.237900  20.34

# Getting objects statistics

In [6]:
objstats = apply_object_stats_df(corrected, magstats)
display(objstats)

,ndethist,ncovhist,mjdstarthist,mjdendhist,meanra,meandec,sigmara,sigmadec,firstmjd,lastmjd,...,nearZTF,nearPS1,stellar,corrected,ndet,ndubious,g-r_max,g-r_max_corr,g-r_mean,g-r_mean_corr
objectId,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaecgt,569.0,1085.0,58101.123472,59009.280961,315.464120,55.418098,0.000085,0.000066,58289.408912,59009.280961,...,True,True,True,True,439,3,2.268444,2.095278,2.126385,2.158688
ZTF18aaioozo,1032.0,1941.0,58218.421956,59009.276736,275.371236,36.742764,0.000067,0.000060,58244.431273,59009.276736,...,True,True,True,True,158,0,-0.458384,-0.047698,0.063620,0.035294
ZTF18aaurrwb,12.0,509.0,58227.365694,58278.362986,256.997487,40.866430,0.000310,0.000133,58258.374213,58278.362986,...,False,False,False,False,3,0,NaN,NaN,NaN,NaN
ZTF18aazeood,1048.0,1893.0,58078.177292,59010.477998,327.181484,49.716779,0.000078,0.000059,58278.410035,59010.477998,...,True,True,True,True,763,0,0.567499,0.648202,0.587791,0.695056
ZTF18abadmxd,5.0,153.0,58281.431609,58320.457442,339.196380,57.996095,0.000121,0.000053,58281.431609,58320.457442,...,True,True,True,True,3,0,NaN,NaN,NaN,NaN
ZTF18abadvod,10.0,479.0,58281.448113,58669.405718,309.249942,34.754976,0.000044,0.000074,58281.448113,58669.405718,...,True,True,True,True,8,0,0.972050,0.719368,0.577728,0.693038
ZTF18abadvtf,1.0,16.0,58281.450579,58281.450579,314.302906,41.961783,NaN,NaN,58281.450579,58281.450579,...,False,False,False,False,1,0,NaN,NaN,NaN,NaN
ZTF18abbklxk,1.0,4.0,58285.333715,58285.333715,274.962513,-6.724415,NaN,NaN,58285.333715,58285.333715,...,True,True,True,True,1,0,NaN,NaN,NaN,NaN
ZTF18abdkytr,5.0,331.0,58301.322708,58639.441701,295.403046,29.808414,0.000022,0.000035,58301.322708,58639.441701,...,False,True,False,False,3,1,0.518068,NaN,0.565115,NaN


# Doing dm/dt

In [7]:
dmdt = do_dmdt_df(magstats, non_detections)
display(dmdt)

close_nondet  dmdt_first  dm_first  sigmadm_first   dt_first
objectId     fid                                                              
ZTF17aaaecgt 1           False         NaN       NaN            NaN        NaN
             2           False   -1.775718 -3.544106     -20.187331   1.958495
ZTF18aaioozo 1           False   -0.172320 -2.113401     -20.575474  11.932870
             2           False   -0.160943 -2.448099     -20.232353  15.016817
ZTF18aaurrwb 1            True    0.001411 -0.166700     -20.735438  25.904178
ZTF18aazeood 1           False   -2.791695 -2.705709     -20.635133   0.959433
             2           False         NaN       NaN            NaN        NaN
ZTF18abadmxd 1           False   -2.276319 -2.303377     -20.627207   0.976701
ZTF18abadvod 1           False   -0.084975 -0.301922     -20.552155   1.000012
             2           False   -0.545535 -0.740147     -20.702034   1.084769
ZTF18abadvtf 2           False   -1.972541 -2.198999     -20.761900   1.086771
ZTF18abbklxk 2           False   -0.156795 -0.454407     -20.017875   1.961030
ZTF18abdkytr 1           False   -1.320684 -1.308931     -19.794093   0.843368
             2           False         NaN       NaN            NaN        NaN
ZTF18abgptvl 1           False   -0.452267 -1.123180     -20.624476   2.099769
             2           False   -0.728268 -0.968462     -19.711232   1.055648

In [8]:
get_clean_corrected(corrected)

,objectId,candid,mjd,fid,pid,diffmaglim,isdiffpos,nid,ra,dec,...,sigmagapbig,rfid,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext,corrected,dubious,has_stamp,parent_candid,step_id_corr
0,ZTF17aaaecgt,550436093915015009,58304.436099,1,550436093915,20.544632,1.0,550,315.464000,55.418125,...,0.0457,801120139,16.510900,0.014875,0.018298,True,False,True,0,corr_bulk_0.0.1
1,ZTF17aaaecgt,1209408473915010011,58963.408472,1,1209408473915,20.269983,-1.0,1209,315.464097,55.418110,...,0.0331,801120139,17.879532,0.055861,0.067333,True,False,True,0,corr_bulk_0.0.1
2,ZTF17aaaecgt,1004203723915015017,58758.203727,1,1004203723915,20.481764,1.0,1004,315.464054,55.418110,...,0.1394,801120139,16.742756,0.009440,0.016224,True,False,True,0,corr_bulk_0.0.1
3,ZTF17aaaecgt,978289493915010013,58732.289491,1,978289493915,20.881487,-1.0,978,315.464145,55.418139,...,0.0298,801120139,17.739686,0.052527,0.062059,True,False,True,0,corr_bulk_0.0.1
4,ZTF17aaaecgt,1009243913915015013,58763.243912,1,1009243913915,20.051744,1.0,1009,315.464014,55.418086,...,0.1871,801120139,16.697498,0.016639,0.020905,True,False,True,0,corr_bulk_0.0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,ZTF18abdkytr,547322701515010698,58301.322708,1,547322701515,20.368393,-1.0,547,295.403046,29.808378,...,0.1221,686120115,15.715557,100.000000,0.012717,True,False,True,0,corr_bulk_0.0.1
1377,ZTF18abdkytr,885441701515010171,58639.441701,1,885441701515,21.037960,-1.0,885,295.403068,29.808417,...,0.1247,686120115,15.709698,100.000000,0.012696,True,False,True,0,corr_bulk_0.0.1
1378,ZTF18abdkytr,862425321515010174,58616.425324,2,862425321515,20.724701,-1.0,862,295.403025,29.808448,...,0.0939,686120115,NaN,NaN,NaN,False,True,False,885441701515010171,corr_bulk_0.0.1
1379,ZTF18abgptvl,558465505215015008,58312.465509,1,558465505215,20.938648,1.0,558,347.973628,58.613658,...,0.3611,804120152,16.117216,100.000000,0.006551,True,False,True,0,corr_bulk_0.0.1
